In [1]:
import os
import json
import torch
import torch.nn as nn
from PIL import Image
from torchvision import transforms
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
print("done")
from models.multimodalTransformer import MultiModalTransformer
print("done1")
from models.GNN import GNN, MultiGNN, getVSG, getVSGInference, getPyVSG, getTSG, getPyTSG, emptyVSG, getCMSG1, getCMSG2, getCMSG3

/home2/siddharthmavani/miniforge3/envs/mfnd/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


done


/home2/siddharthmavani/miniforge3/envs/mfnd/lib/python3.8/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


Transformers will be trained on GPU
done1


/home2/siddharthmavani/miniforge3/envs/mfnd/lib/python3.8/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


VSG Using device: cuda
GNN running on GPU


In [2]:
if (torch.cuda.is_available()):
    device = torch.device("cuda")
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [3]:
class MultiModalClassifier(nn.Module):
    # 768, 8, 2048, 0.1, GNN_DIM
    def __init__(self, d_model, num_heads, d_ff, dropout, gnn_dim):
        super(MultiModalClassifier, self).__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_ff = d_ff
        self.dropout = dropout
        
        self.transformer = MultiModalTransformer(d_model, num_heads, d_ff, dropout)
        self.GNN = GNN(gnn_dim)
        self.classifier = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(d_model+64, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128,32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )
        
    def forward(self, text_batch, image, data_visual):
        x = self.transformer(text_batch, image)
        x_graph = self.GNN(data_visual)
        
        # Concatenate the outputs
        x_combined = torch.cat((x, x_graph), dim=1)
        
        x = self.classifier(x_combined)
        return x


transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()              
])

In [4]:
# loading the model state dicts
def load_model(model, modelPath):
    model.load_state_dict(torch.load(modelPath))
    return model

In [5]:
NODE_EMBEDDING_SIZE = 100
GNN_DIM = NODE_EMBEDDING_SIZE
model = MultiModalClassifier(768, 8, 2048, 0.1, GNN_DIM)
model = load_model(model, "./elections-1.pth")
model.to(device)

MultiModalClassifier(
  (transformer): MultiModalTransformer(
    (encoder_layer): EncoderLayer(
      (self_attn): MultiHeadAttention(
        (W_q): Linear(in_features=768, out_features=768, bias=True)
        (W_k): Linear(in_features=768, out_features=768, bias=True)
        (W_v): Linear(in_features=768, out_features=768, bias=True)
        (W_o): Linear(in_features=768, out_features=768, bias=True)
      )
      (feed_forward): PositionWiseFeedForward(
        (fc1): Linear(in_features=768, out_features=2048, bias=True)
        (fc2): Linear(in_features=2048, out_features=768, bias=True)
        (relu): ReLU()
      )
      (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (patch_embedding): PatchEmbedding(
      (projection): Sequential(
        (0): Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1=16, p2=16)
        (1): Linear(in_feat

In [6]:
def MFND_Inference(text, ImgPath):

    threshold = 0.5
    length = 10001
    placeholder_image = torch.zeros((1, 3, 256, 256))

    # Loading the Image
    if os.path.exists(ImgPath):
        try:
            image = Image.open(ImgPath)
            if(image.mode != 'RGB'):
                    image = image.convert('RGB')
            image = transform(image)
            image = image.unsqueeze(0)
        except:
            image = placeholder_image
    else:
        return "Image not found"          


    # Getting VSG
    try:
        vsg = getVSGInference(ImgPath)
        data_vsg = getPyVSG(vsg)
    except Exception as e:
        print(f"Error processing image file: {str(e)}")
        print("Taking Empty VSG")
        vsg = emptyVSG()
        data_vsg = getPyVSG(vsg)


    data_vsg = data_vsg.to(device)
    
    output = model([text], image, data_vsg)

    return output

     
    

In [7]:
print(MFND_Inference("This is a sample text", "/home2/siddharthmavani/elections/images/hindi-971.jpg"))

Image not found
